In [10]:
from pymongo import MongoClient
import json
from collections import Counter

In [11]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
posts = db['posts']

In [12]:
source_url = {'https://www.reddit.com/r/dataisugly', 'https://badvisualisations.tumblr.com/', 'https://viz.wtf/'}

In [13]:
output = [p for p in posts.find({})
              if p['source_url'] in source_url and
                 p['preview']['url'] != '' and
                 p['thumbnail']['url'] != '' and
                 p['media_type'] == 'image'
         ]

In [14]:
len(output)

6589

In [15]:
for o in output:
    del o['_id']

In [16]:
json.dump(output[0:1000], open('./output.json', 'w'))

In [24]:
c = Counter([t for p in posts.find() for t in p['tags']])

In [25]:
c.most_common()[:50]

[('WTFViz', 2119),
 ('Percentages', 985),
 ('BarChart', 705),
 ('submission', 704),
 ('PieChart', 477),
 ('DonutChart', 373),
 ('3D', 304),
 ('Radial', 276),
 ('Bad Scale', 231),
 ('Diagram', 162),
 ('Map', 156),
 ('Pie Gore', 150),
 ('LabelLines', 147),
 ('Clusterfuck', 139),
 ('LineChart', 131),
 ('OC', 113),
 ('What the Fuck?', 94),
 ('Iconography', 88),
 ('StackedBarChart', 84),
 ('Agendas Gone Wild', 81),
 ('Timeline', 81),
 ('Wedges', 70),
 ('Arrows', 69),
 ('BubbleChart', 67),
 ('CoxcombChart', 63),
 ('Euler', 61),
 ('R2: Nothing wrong', 55),
 ('data visualisation', 50),
 ('AreaChart', 45),
 ('data visualization', 42),
 ('NotPartToWhole', 35),
 ('Venn', 32),
 ('R1: Parody', 31),
 ('bar graph', 29),
 ('Rainbow', 28),
 ('Area/Volume', 27),
 ('Clock', 26),
 ('3D is Cool!', 25),
 ('NotPartWhole', 25),
 ('R3: Dataisbeautiful', 20),
 ('Flawed Flows', 19),
 ('PartToWhole', 19),
 ('Scatterplot', 18),
 ('NetworkDiagram', 16),
 ('NotPeriodic', 16),
 ('TriangleChart', 15),
 ('Dial', 15),
 

In [26]:
c = Counter([p['author'] for p in posts.find() if 'author' in p])

In [27]:
c.most_common()[:50]

[('', 1374),
 ('rhiever', 80),
 ('CJMinard', 58),
 ('minimaxir', 36),
 ('zonination', 34),
 ('[deleted]', 31),
 ('akeemtheafricandream', 28),
 ('NelsonMinar', 24),
 ('jensyao', 20),
 ('confused_confucius07', 19),
 ('ProgVal', 17),
 ('OllyTwist', 16),
 ('Renegade_Meister', 15),
 ('zevobh', 14),
 ('denacioust', 14),
 ('Epistaxis', 12),
 ('almodozo', 11),
 ('misnamed', 11),
 ('suity1', 10),
 ('WE_THEPEOPLE', 10),
 ('phil_blumpkin', 10),
 ('wspaniel', 10),
 ('SymphMeta', 10),
 ('iqueerified', 9),
 ('WHY_DO_I_SHOUT', 8),
 ('crvc', 8),
 ('Thefriendlyfaceplant', 8),
 ('MatthewLaw', 8),
 ('Batrachus', 8),
 ('colako', 8),
 ('pongpingpongpingpong', 7),
 ('Catharrrsis', 7),
 ('DiamondMinah', 7),
 ('nate_silver_surfer', 7),
 ('Curran919', 7),
 ('toot_ma_boot', 7),
 ('Null_HHockey', 7),
 ('mrdelayer', 7),
 ('ar_604', 6),
 ('wheresflateric', 6),
 ('ofsinope', 6),
 ('Reddits_Worst_Night', 6),
 ('rocketman0739', 6),
 ('--DrScience--', 6),
 ('bubbles212', 6),
 ('mrgann', 6),
 ('owmyfoothurts', 6),
 ('v